In [ ]:
import pandas as pd
from pangaeapy.pandataset import PanDataSet

pds = PanDataSet(897888)

header = {}
for k, event in enumerate(pds.events):
    header.update(
        {
            k: {
                "longitude": event.longitude,
                "latitude": event.latitude,
                "elevation": event.elevation,
                "event": event.label,
            },
        },
    )

df = pds.data
pdf = pd.DataFrame.from_dict(header).T

In [ ]:
cabo_frio = [
    "HAB_A01",
    "HAB_A02",
    "HAB_A03",
    "HAB_A04",
    "HAB_B1",
    "HAB_B2",
]


pdf = pdf[~pdf["event"].isin(cabo_frio)]
df = df[~df["Event"].isin(cabo_frio)]

In [ ]:
df.loc[:, "SST_UK37"] = (df["UK'37"] - 0.069) / 0.033

In [ ]:
bbox = (
    pdf["longitude"].min(),
    pdf["longitude"].max(),
    pdf["latitude"].min(),
    pdf["latitude"].max(),
)

In [ ]:
from oceans.datasets import woa_subset

ds = woa_subset(
    *bbox,
    variable="temperature",
    time_period="annual",
    resolution="1/4",
)

ds = ds.sel(depth=0)

In [ ]:
%%time

ds.compute()

In [ ]:
woa = {}

for k, row in pdf.iterrows():
    tmp = ds.sel(
        {"lon": row["longitude"], "lat": row["latitude"]},
        method="nearest",
    )

    woa.update(
        {
            row["event"]: {
                "woa": tmp["t_mn"].to_numpy().squeeze().tolist(),
                "lon": tmp["lon"].to_numpy().squeeze().tolist(),
                "lat": tmp["lat"].to_numpy().squeeze().tolist(),
                "lon_obs": row["longitude"],
                "lat_obs": row["latitude"],
            }
        }
    )

In [ ]:
import seawater as sw

w = pd.DataFrame(woa).T

lats = list(zip(w["lat"], w["lat_obs"]))
lons = list(zip(w["lon"], w["lon_obs"]))
dists = [sw.dist(*location)[0][0] for location in zip(lats, lons)]
w["dist"] = dists

In [ ]:
SSTs = pd.concat(
    [
        df[["SST (1-12)", "Event", "SST_UK37"]].reset_index(drop=True),
        w.reset_index()
    ],
    axis=1,
).dropna()

SSTs.tail()

In [ ]:
import statsmodels.api as sm


Y = SSTs["SST_UK37"]
x = SSTs["woa"]

res = sm.OLS(
    Y,
    x,
).fit()

print(res.summary())

In [ ]:
import matplotlib.pyplot as plt


lims = [17.5, 28.5, 17.5, 28.5]
fig, ax = plt.subplots(figsize=(5, 5))
l = ax.plot(
    x,
    Y,
    marker="o",
    markerfacecolor="blue",
    markeredgecolor="w",
    linestyle="none",
    alpha=0.95,
    markersize=8,
)


ax.axis(lims);